In [1]:
from itertools import permutations
from collections import deque

# Set n
n = 4
initial_perm = tuple(range(1, n + 1))

# Generate all permutations
all_perms = list(permutations(range(1, n + 1))) # with n = 4 there are 24 permutations

# Assign unique index to each permutation for easy lookup
perm_to_index = {perm: idx for idx, perm in enumerate(all_perms)}

# Initialize distance dictionary D
X = 99999999  # Large integer
D = [X] * len(all_perms)
D[perm_to_index[initial_perm]] = 0

# Queue for BFS
queue = deque()
queue.append(initial_perm)

# BFS for adjacent translocation distance
while queue:
    current = queue.popleft()
    current_dist = D[perm_to_index[current]]

    for i in range(n - 1):
        # Perform adjacent translocation (swap i and i+1)
        new_perm = list(current)
        new_perm[i], new_perm[i + 1] = new_perm[i + 1], new_perm[i]
        new_perm = tuple(new_perm)

        if D[perm_to_index[new_perm]] == X:
            D[perm_to_index[new_perm]] = current_dist + 1
            queue.append(new_perm)

# Display results
print("Permutation : Minimum Adjacent Translocation Distance from (1,2,3,4)")
for perm, idx in perm_to_index.items():
    print(f"{perm} : {D[idx]}")


Permutation : Minimum Adjacent Translocation Distance from (1,2,3,4)
(1, 2, 3, 4) : 0
(1, 2, 4, 3) : 1
(1, 3, 2, 4) : 1
(1, 3, 4, 2) : 2
(1, 4, 2, 3) : 2
(1, 4, 3, 2) : 3
(2, 1, 3, 4) : 1
(2, 1, 4, 3) : 2
(2, 3, 1, 4) : 2
(2, 3, 4, 1) : 3
(2, 4, 1, 3) : 3
(2, 4, 3, 1) : 4
(3, 1, 2, 4) : 2
(3, 1, 4, 2) : 3
(3, 2, 1, 4) : 3
(3, 2, 4, 1) : 4
(3, 4, 1, 2) : 4
(3, 4, 2, 1) : 5
(4, 1, 2, 3) : 3
(4, 1, 3, 2) : 4
(4, 2, 1, 3) : 4
(4, 2, 3, 1) : 5
(4, 3, 1, 2) : 5
(4, 3, 2, 1) : 6


In [33]:
from collections import deque

def optimized_interval_adjacent_translocation(start, target):
    n = len(start)
    visited = {start: None}
    queue = deque([start])

    while queue:
        perm = queue.popleft()
        if perm == target:
            break

        for i in range(n):
            for j in range(i, n):
                interval = perm[i:j + 1]
                rest = perm[:i] + perm[j + 1:]

                for k in range(len(rest) + 1):
                    if k != i:
                        new_perm = rest[:k] + interval + rest[k:]
                        if new_perm not in visited:
                            visited[new_perm] = perm
                            queue.append(new_perm)

    # Reconstruct the path
    path = []
    current = target
    while current:
        path.append(current)
        current = visited[current]
    path.reverse()

    return path

# Example usage
start_perm = (1, 2, 3, 4, 5, 6,7,8)
target_perm = (8,7,6, 5, 4, 3, 2, 1)
optimal_path = optimized_interval_adjacent_translocation(start_perm, target_perm)

print("Optimal path of translocations:")
for step, perm in enumerate(optimal_path):
    print(f"Step {step}: {perm}")

print(f"\nMinimum translocations from {start_perm} to {target_perm}: {len(optimal_path) - 1}")


Optimal path of translocations:
Step 0: (1, 2, 3, 4, 5, 6, 7, 8)
Step 1: (2, 1, 3, 4, 5, 6, 7, 8)
Step 2: (4, 5, 6, 2, 1, 3, 7, 8)
Step 3: (4, 3, 7, 8, 5, 6, 2, 1)
Step 4: (8, 5, 4, 3, 7, 6, 2, 1)
Step 5: (8, 7, 6, 5, 4, 3, 2, 1)

Minimum translocations from (1, 2, 3, 4, 5, 6, 7, 8) to (8, 7, 6, 5, 4, 3, 2, 1): 5


In [27]:
from itertools import permutations
from collections import deque
import heapq

def generate_single_translocation(perm, start_idx, end_idx, insert_pos):
    """
    Generate a single translocation: move interval [start_idx:end_idx] to insert_pos.
    Returns the resulting permutation.
    """
    # Extract the interval (keeping its internal order)
    interval = perm[start_idx:end_idx]
    # Get the remaining elements
    remaining = perm[:start_idx] + perm[end_idx:]
    # Insert interval at the specified position
    result = remaining[:insert_pos] + interval + remaining[insert_pos:]
    return result

def get_all_possible_translocations(perm):
    """
    Get all possible single translocations from a given permutation.
    Returns list of (result_permutation, move_info) tuples.
    """
    translocations = []
    n = len(perm)
    
    # Try all possible intervals
    for start_idx in range(n):
        for end_idx in range(start_idx + 1, n + 1):
            interval = perm[start_idx:end_idx]
            remaining = perm[:start_idx] + perm[end_idx:]
            
            # Try all possible insertion positions
            for insert_pos in range(len(remaining) + 1):
                result = remaining[:insert_pos] + interval + remaining[insert_pos:]
                
                # Only include if different from original
                if result != perm:
                    move_info = {
                        'interval': interval,
                        'from_pos': (start_idx, end_idx),
                        'to_pos': insert_pos,
                        'description': f"Move '{interval}' from [{start_idx}:{end_idx}] to pos {insert_pos}"
                    }
                    translocations.append((result, move_info))
    
    return translocations

def heuristic_distance(current_perm, target_perm):
    """
    Heuristic function: number of elements not in correct position.
    This is admissible (never overestimates) because each translocation
    can fix at most n-1 positions in the best case.
    """
    return sum(1 for i in range(len(current_perm)) if current_perm[i] != target_perm[i])

def better_heuristic(current_perm, target_perm):
    """
    Better heuristic: minimum number of intervals that need to be moved.
    More informed than simple position counting.
    """
    if current_perm == target_perm:
        return 0
    
    n = len(current_perm)
    # Count longest common subsequences that are already in place
    correctly_placed = 0
    
    # Find the longest subsequence that's already in the right relative positions
    target_positions = {val: i for i, val in enumerate(target_perm)}
    current_positions = {val: i for i, val in enumerate(current_perm)}
    
    # Simple heuristic: count elements that are out of place
    out_of_place = sum(1 for val in current_perm 
                      if current_positions[val] != target_positions[val])
    
    # Each move can potentially fix multiple positions, but conservatively
    # estimate that we need at least ceil(out_of_place / n) moves
    return max(1, out_of_place // n) if out_of_place > 0 else 0

class SearchNode:
    def __init__(self, permutation, path, moves, g_cost, h_cost):
        self.permutation = permutation
        self.path = path  # List of permutations in the path
        self.moves = moves  # List of move descriptions
        self.g_cost = g_cost  # Actual cost (number of moves so far)
        self.h_cost = h_cost  # Heuristic cost (estimated remaining cost)
        self.f_cost = g_cost + h_cost  # Total estimated cost
    
    def __lt__(self, other):
        # For priority queue: prefer lower f_cost, then lower g_cost
        if self.f_cost == other.f_cost:
            return self.g_cost < other.g_cost
        return self.f_cost < other.f_cost

def branch_and_bound_search(start_perm, target_perm, max_depth=10, use_better_heuristic=True):
    """
    Find shortest path using branch and bound with A* strategy.
    """
    if start_perm == target_perm:
        return 0, [start_perm], []
    
    # Choose heuristic function
    heuristic_func = better_heuristic if use_better_heuristic else heuristic_distance
    
    # Priority queue: (f_cost, node_id, node)
    initial_h = heuristic_func(start_perm, target_perm)
    initial_node = SearchNode(start_perm, [start_perm], [], 0, initial_h)
    
    priority_queue = [initial_node]
    heapq.heapify(priority_queue)
    
    visited = {start_perm: 0}  # permutation -> best_g_cost_so_far
    best_solution = None
    nodes_explored = 0
    
    print(f"Starting branch and bound search...")
    print(f"Initial heuristic estimate: {initial_h}")
    
    while priority_queue:
        current_node = heapq.heappop(priority_queue)
        nodes_explored += 1
        
        # Pruning: if we already found a better solution, skip
        if best_solution and current_node.f_cost >= best_solution.g_cost:
            continue
            
        # Depth limiting
        if current_node.g_cost >= max_depth:
            continue
        
        # Progress reporting
        if nodes_explored % 1000 == 0:
            print(f"Explored {nodes_explored} nodes, current best f_cost: {current_node.f_cost}")
        
        # Generate all possible next moves
        possible_moves = get_all_possible_translocations(current_node.permutation)
        
        for next_perm, move_info in possible_moves:
            new_g_cost = current_node.g_cost + 1
            
            # Check if we reached the target
            if next_perm == target_perm:
                solution_node = SearchNode(
                    next_perm,
                    current_node.path + [next_perm],
                    current_node.moves + [move_info['description']],
                    new_g_cost,
                    0
                )
                
                if not best_solution or new_g_cost < best_solution.g_cost:
                    best_solution = solution_node
                    print(f"Found solution with cost {new_g_cost}!")
                continue
            
            # Pruning: skip if we've seen this state with better or equal cost
            if next_perm in visited and visited[next_perm] <= new_g_cost:
                continue
            
            # Pruning: skip if estimated total cost exceeds current best
            new_h_cost = heuristic_func(next_perm, target_perm)
            new_f_cost = new_g_cost + new_h_cost
            
            if best_solution and new_f_cost >= best_solution.g_cost:
                continue
            
            # Add to queue
            visited[next_perm] = new_g_cost
            new_node = SearchNode(next_perm, 
                                current_node.path + [next_perm],
                                current_node.moves + [move_info['description']],
                                new_g_cost, 
                                new_h_cost)
            
            heapq.heappush(priority_queue, new_node)
    
    print(f"Search completed. Explored {nodes_explored} nodes.")
    
    if best_solution:
        return best_solution.g_cost, best_solution.path, best_solution.moves
    else:
        return float('inf'), [], []

def analyze_with_branch_and_bound(start_perm, target_perm, max_depth=8):
    """
    Analyze transformation using branch and bound.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"BRANCH AND BOUND ANALYSIS: {start_str} → {target_str}")
    print("=" * 60)
    
    # Try with better heuristic first
    print("Using better heuristic...")
    distance, path, moves = branch_and_bound_search(start_perm, target_perm, max_depth, True)
    
    if distance == float('inf'):
        print(f"No solution found within depth limit {max_depth}")
        
        # Try with simple heuristic as backup
        print("\nTrying with simple heuristic...")
        distance, path, moves = branch_and_bound_search(start_perm, target_perm, max_depth, False)
    
    if distance == float('inf'):
        print("No solution found with either heuristic!")
        return
    
    print(f"\nMinimum distance: {distance}")
    print("\nOptimal transformation path:")
    
    for i, perm in enumerate(path):
        perm_str = ''.join(map(str, perm))
        if i == 0:
            print(f"Step {i}: {perm_str} (start)")
        else:
            print(f"Step {i}: {perm_str}")
            print(f"       <- {moves[i-1]}")
    
    return distance, path, moves

def compare_algorithms(start_perm, target_perm):
    """
    Compare BFS vs Branch and Bound
    """
    print("ALGORITHM COMPARISON")
    print("=" * 60)
    
    # BFS (from previous implementation)
    print("1. BFS (Breadth-First Search):")
    try:
        from collections import deque
        # Simple BFS for comparison
        if start_perm == target_perm:
            bfs_distance = 0
        else:
            queue = deque([(start_perm, 0)])
            visited = {start_perm}
            bfs_distance = float('inf')
            
            while queue and bfs_distance == float('inf'):
                current_perm, depth = queue.popleft()
                if depth >= 6:  # Limit for comparison
                    break
                    
                moves = get_all_possible_translocations(current_perm)
                for next_perm, _ in moves:
                    if next_perm == target_perm:
                        bfs_distance = depth + 1
                        break
                    if next_perm not in visited:
                        visited.add(next_perm)
                        queue.append((next_perm, depth + 1))
        
        print(f"   Result: {bfs_distance}")
    except:
        print("   BFS failed")
    
    print("\n2. Branch and Bound with A*:")
    bb_distance, _, _ = branch_and_bound_search(start_perm, target_perm, 6, True)
    print(f"   Result: {bb_distance}")

if __name__ == "__main__":
    # Test cases
    test_cases = [
        ((1, 2, 3), (2, 1, 3)),  # Simple case
        ((1, 2, 3, 4), (2, 3, 4, 1)),  # Should be 1 step
        ((1, 2, 3, 4), (4, 3, 2, 1)),  # Reverse of length 4
        ((1, 2, 3, 4, 5), (5, 4, 3, 2, 1)),  # The problematic case
        ((1, 2, 3, 4, 5,6,7), (7,6,5, 4, 3, 2, 1)),  # The problematic case
    ]
    
    for i, (start, target) in enumerate(test_cases):
        print(f"\nTEST CASE {i+1}:")
        analyze_with_branch_and_bound(start, target)
        print("\n" + "="*60)
    
    # Compare algorithms on the problematic case
    print("\nCOMPARISON ON PROBLEMATIC CASE:")
    compare_algorithms((1, 2, 3, 4, 5), (5, 4, 3, 2, 1))


TEST CASE 1:
BRANCH AND BOUND ANALYSIS: 123 → 213
Using better heuristic...
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 1!
Search completed. Explored 1 nodes.

Minimum distance: 1

Optimal transformation path:
Step 0: 123 (start)
Step 1: 213
       <- Move '(1,)' from [0:1] to pos 1


TEST CASE 2:
BRANCH AND BOUND ANALYSIS: 1234 → 2341
Using better heuristic...
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 1!
Search completed. Explored 3 nodes.

Minimum distance: 1

Optimal transformation path:
Step 0: 1234 (start)
Step 1: 2341
       <- Move '(1,)' from [0:1] to pos 3


TEST CASE 3:
BRANCH AND BOUND ANALYSIS: 1234 → 4321
Using better heuristic...
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 3!
Search completed. Explored 23 nodes.

Minimum distance: 3

Optimal transformation path:
Step 0: 1234 (start)
Step 1: 2134
       <- Move '(1,)' from [0:1] to

In [30]:
from itertools import permutations
from collections import deque
import heapq

def generate_single_translocation(perm, start_idx, end_idx, insert_pos):
    """
    Generate a single translocation: move interval [start_idx:end_idx] to insert_pos.
    Returns the resulting permutation.
    """
    # Extract the interval (keeping its internal order)
    interval = perm[start_idx:end_idx]
    # Get the remaining elements
    remaining = perm[:start_idx] + perm[end_idx:]
    # Insert interval at the specified position
    result = remaining[:insert_pos] + interval + remaining[insert_pos:]
    return result

def get_all_possible_translocations(perm):
    """
    Get all possible single translocations from a given permutation.
    Returns list of (result_permutation, move_info) tuples.
    """
    translocations = []
    n = len(perm)
    
    # Try all possible intervals
    for start_idx in range(n):
        for end_idx in range(start_idx + 1, n + 1):
            interval = perm[start_idx:end_idx]
            remaining = perm[:start_idx] + perm[end_idx:]
            
            # Try all possible insertion positions
            for insert_pos in range(len(remaining) + 1):
                result = remaining[:insert_pos] + interval + remaining[insert_pos:]
                
                # Only include if different from original
                if result != perm:
                    move_info = {
                        'interval': interval,
                        'from_pos': (start_idx, end_idx),
                        'to_pos': insert_pos,
                        'description': f"Move '{interval}' from [{start_idx}:{end_idx}] to pos {insert_pos}"
                    }
                    translocations.append((result, move_info))
    
    return translocations

def heuristic_distance(current_perm, target_perm):
    """
    Heuristic function: number of elements not in correct position.
    This is admissible (never overestimates) because each translocation
    can fix at most n-1 positions in the best case.
    """
    return sum(1 for i in range(len(current_perm)) if current_perm[i] != target_perm[i])

def better_heuristic(current_perm, target_perm):
    """
    Better heuristic: minimum number of intervals that need to be moved.
    More informed than simple position counting.
    """
    if current_perm == target_perm:
        return 0
    
    n = len(current_perm)
    # Count longest common subsequences that are already in place
    correctly_placed = 0
    
    # Find the longest subsequence that's already in the right relative positions
    target_positions = {val: i for i, val in enumerate(target_perm)}
    current_positions = {val: i for i, val in enumerate(current_perm)}
    
    # Simple heuristic: count elements that are out of place
    out_of_place = sum(1 for val in current_perm 
                      if current_positions[val] != target_positions[val])
    
    # Each move can potentially fix multiple positions, but conservatively
    # estimate that we need at least ceil(out_of_place / n) moves
    return max(1, out_of_place // n) if out_of_place > 0 else 0

class SearchNode:
    def __init__(self, permutation, path, moves, g_cost, h_cost):
        self.permutation = permutation
        self.path = path  # List of permutations in the path
        self.moves = moves  # List of move descriptions
        self.g_cost = g_cost  # Actual cost (number of moves so far)
        self.h_cost = h_cost  # Heuristic cost (estimated remaining cost)
        self.f_cost = g_cost + h_cost  # Total estimated cost
    
    def __lt__(self, other):
        # For priority queue: prefer lower f_cost, then lower g_cost
        if self.f_cost == other.f_cost:
            return self.g_cost < other.g_cost
        return self.f_cost < other.f_cost

def branch_and_bound_search(start_perm, target_perm, max_depth=10, use_better_heuristic=True):
    """
    Find shortest path using branch and bound with A* strategy.
    """
    if start_perm == target_perm:
        return 0, [start_perm], []
    
    # Choose heuristic function
    heuristic_func = better_heuristic if use_better_heuristic else heuristic_distance
    
    # Priority queue: (f_cost, node_id, node)
    initial_h = heuristic_func(start_perm, target_perm)
    initial_node = SearchNode(start_perm, [start_perm], [], 0, initial_h)
    
    priority_queue = [initial_node]
    heapq.heapify(priority_queue)
    
    visited = {start_perm: 0}  # permutation -> best_g_cost_so_far
    best_solution = None
    nodes_explored = 0
    
    print(f"Starting branch and bound search...")
    print(f"Initial heuristic estimate: {initial_h}")
    
    while priority_queue:
        current_node = heapq.heappop(priority_queue)
        nodes_explored += 1
        
        # Pruning: if we already found a better solution, skip
        if best_solution and current_node.f_cost >= best_solution.g_cost:
            continue
            
        # Depth limiting
        if current_node.g_cost >= max_depth:
            continue
        
        # Progress reporting
        if nodes_explored % 1000 == 0:
            print(f"Explored {nodes_explored} nodes, current best f_cost: {current_node.f_cost}")
        
        # Generate all possible next moves
        possible_moves = get_all_possible_translocations(current_node.permutation)
        
        for next_perm, move_info in possible_moves:
            new_g_cost = current_node.g_cost + 1
            
            # Check if we reached the target
            if next_perm == target_perm:
                solution_node = SearchNode(
                    next_perm,
                    current_node.path + [next_perm],
                    current_node.moves + [move_info['description']],
                    new_g_cost,
                    0
                )
                
                if not best_solution or new_g_cost < best_solution.g_cost:
                    best_solution = solution_node
                    print(f"Found solution with cost {new_g_cost}!")
                continue
            
            # Pruning: skip if we've seen this state with better or equal cost
            if next_perm in visited and visited[next_perm] <= new_g_cost:
                continue
            
            # Pruning: skip if estimated total cost exceeds current best
            new_h_cost = heuristic_func(next_perm, target_perm)
            new_f_cost = new_g_cost + new_h_cost
            
            if best_solution and new_f_cost >= best_solution.g_cost:
                continue
            
            # Add to queue
            visited[next_perm] = new_g_cost
            new_node = SearchNode(next_perm, 
                                current_node.path + [next_perm],
                                current_node.moves + [move_info['description']],
                                new_g_cost, 
                                new_h_cost)
            
            heapq.heappush(priority_queue, new_node)
    
    print(f"Search completed. Explored {nodes_explored} nodes.")
    
    if best_solution:
        return best_solution.g_cost, best_solution.path, best_solution.moves
    else:
        return float('inf'), [], []

def analyze_with_branch_and_bound(start_perm, target_perm, max_depth=8):
    """
    Analyze transformation using branch and bound.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"BRANCH AND BOUND ANALYSIS: {start_str} → {target_str}")
    print("=" * 60)
    
    # Try with better heuristic first
    print("Using better heuristic...")
    distance, path, moves = branch_and_bound_search(start_perm, target_perm, max_depth, True)
    
    if distance == float('inf'):
        print(f"No solution found within depth limit {max_depth}")
        
        # Try with simple heuristic as backup
        print("\nTrying with simple heuristic...")
        distance, path, moves = branch_and_bound_search(start_perm, target_perm, max_depth, False)
    
    if distance == float('inf'):
        print("No solution found with either heuristic!")
        return
    
    print(f"\nMinimum distance: {distance}")
    print("\nOptimal transformation path:")
    
    for i, perm in enumerate(path):
        perm_str = ''.join(map(str, perm))
        if i == 0:
            print(f"Step {i}: {perm_str} (start)")
        else:
            print(f"Step {i}: {perm_str}")
            print(f"       <- {moves[i-1]}")
    
    return distance, path, moves

def compare_algorithms(start_perm, target_perm):
    """
    Compare BFS vs Branch and Bound
    """
    print("ALGORITHM COMPARISON")
    print("=" * 60)
    
    # BFS (from previous implementation)
    print("1. BFS (Breadth-First Search):")
    try:
        from collections import deque
        # Simple BFS for comparison
        if start_perm == target_perm:
            bfs_distance = 0
        else:
            queue = deque([(start_perm, 0)])
            visited = {start_perm}
            bfs_distance = float('inf')
            
            while queue and bfs_distance == float('inf'):
                current_perm, depth = queue.popleft()
                if depth >= 6:  # Limit for comparison
                    break
                    
                moves = get_all_possible_translocations(current_perm)
                for next_perm, _ in moves:
                    if next_perm == target_perm:
                        bfs_distance = depth + 1
                        break
                    if next_perm not in visited:
                        visited.add(next_perm)
                        queue.append((next_perm, depth + 1))
        
        print(f"   Result: {bfs_distance}")
    except:
        print("   BFS failed")
    
    print("\n2. Branch and Bound with A*:")
    bb_distance, _, _ = branch_and_bound_search(start_perm, target_perm, 6, True)
    print(f"   Result: {bb_distance}")

def generate_random_test_cases(n=5, num_cases=20, seed=42):
    """
    Generate random test cases for permutations of size n.
    """
    import random
    random.seed(seed)
    
    all_perms = list(permutations(range(1, n + 1)))
    test_cases = []
    
    for _ in range(num_cases):
        start = random.choice(all_perms)
        target = random.choice(all_perms)
        # Avoid trivial cases where start == target
        while target == start:
            target = random.choice(all_perms)
        test_cases.append((start, target))
    
    return test_cases

def comprehensive_analysis_size5():
    """
    Comprehensive analysis of all permutation pairs of size 5.
    Warning: This is 5! × 5! = 14,400 pairs, so we'll sample smartly.
    """
    print("COMPREHENSIVE ANALYSIS FOR SIZE 5 PERMUTATIONS")
    print("=" * 60)
    
    all_perms = list(permutations(range(1, 6)))  # All 120 permutations of size 5
    print(f"Total permutations of size 5: {len(all_perms)}")
    print(f"Total possible pairs: {len(all_perms) * (len(all_perms) - 1)}")
    
    # Statistics tracking
    distance_counts = {}
    impossible_count = 0
    total_tested = 0
    max_depth_limit = 6
    
    print(f"\nTesting all pairs with depth limit {max_depth_limit}...")
    print("(This may take a while...)")
    
    # Test all pairs (or sample if too many)
    sample_size = min(1000, len(all_perms) * (len(all_perms) - 1))  # Limit for practicality
    
    import random
    random.seed(42)
    
    # Generate sample pairs
    test_pairs = []
    for start in all_perms[:20]:  # Take first 20 as starting points
        for target in all_perms:
            if start != target:
                test_pairs.append((start, target))
    
    # Shuffle and take sample
    random.shuffle(test_pairs)
    test_pairs = test_pairs[:sample_size]
    
    print(f"Testing {len(test_pairs)} randomly sampled pairs...")
    
    for i, (start, target) in enumerate(test_pairs):
        if i % 100 == 0:
            print(f"Progress: {i}/{len(test_pairs)}")
        
        distance, _, _ = branch_and_bound_search(start, target, max_depth_limit, True)
        total_tested += 1
        
        if distance == float('inf'):
            impossible_count += 1
        else:
            distance_counts[distance] = distance_counts.get(distance, 0) + 1
    
    # Report statistics
    print(f"\nRESULTS FROM {total_tested} TEST CASES:")
    print("-" * 40)
    print(f"Impossible within depth {max_depth_limit}: {impossible_count}")
    
    print("\nDistance distribution:")
    for dist in sorted(distance_counts.keys()):
        count = distance_counts[dist]
        percentage = (count / total_tested) * 100
        print(f"Distance {dist}: {count} cases ({percentage:.1f}%)")
    
    return distance_counts, impossible_count

def analyze_specific_patterns():
    """
    Analyze specific interesting patterns in size 5 permutations.
    """
    print("ANALYZING SPECIFIC PATTERNS FOR SIZE 5")
    print("=" * 60)
    
    patterns = [
        # Identity and simple swaps
        ((1, 2, 3, 4, 5), (1, 2, 3, 4, 5)),  # Identity (should be 0)
        ((1, 2, 3, 4, 5), (2, 1, 3, 4, 5)),  # Adjacent swap
        ((1, 2, 3, 4, 5), (1, 3, 2, 4, 5)),  # Non-adjacent swap
        
        # Rotations
        ((1, 2, 3, 4, 5), (2, 3, 4, 5, 1)),  # Left rotation
        ((1, 2, 3, 4, 5), (5, 1, 2, 3, 4)),  # Right rotation
        
        # Reversals
        ((1, 2, 3, 4, 5), (5, 4, 3, 2, 1)),  # Complete reversal
        ((1, 2, 3, 4, 5), (3, 2, 1, 4, 5)),  # Partial reversal
        
        # Complex patterns
        ((1, 2, 3, 4, 5), (3, 1, 4, 2, 5)),  # Complex rearrangement
        ((1, 2, 3, 4, 5), (4, 5, 1, 2, 3)),  # Block movement
    ]
    
    for i, (start, target) in enumerate(patterns):
        start_str = ''.join(map(str, start))
        target_str = ''.join(map(str, target))
        print(f"\nPattern {i+1}: {start_str} → {target_str}")
        
        if start == target:
            print("Distance: 0 (identity)")
            continue
            
        distance, path, moves = branch_and_bound_search(start, target, 8, True)
        print(f"Distance: {distance}")
        
        if distance != float('inf') and distance <= 4:  # Show path for short distances
            print("Path:")
            for j, perm in enumerate(path):
                perm_str = ''.join(map(str, perm))
                if j == 0:
                    print(f"  Step {j}: {perm_str}")
                else:
                    print(f"  Step {j}: {perm_str} <- {moves[j-1]}")

if __name__ == "__main__":
    # # Choose what to run:
    # print("ADJACENT INTERVAL TRANSLOCATION ANALYSIS")
    # print("=" * 60)
    
    # # Option 1: Run specific interesting patterns
    # print("1. ANALYZING SPECIFIC PATTERNS:")
    # analyze_specific_patterns()
    
    # print("\n" + "=" * 60)
    
    # # Option 2: Generate and test random cases
    # print("2. RANDOM TEST CASES:")
    # random_cases = generate_random_test_cases(n=5, num_cases=12, seed=42)
    
    # for i, (start, target) in enumerate(random_cases):
    #     start_str = ''.join(map(str, start))
    #     target_str = ''.join(map(str, target))
    #     print(f"\nRandom Case {i+1}: {start_str} → {target_str}")
        
    #     distance, _, _ = branch_and_bound_search(start, target, 6, True)
    #     print(f"Distance: {distance}")
    
    # Option 3: Comprehensive analysis (uncomment if you want full analysis)
    print("\n" + "=" * 60)
    print("3. COMPREHENSIVE ANALYSIS:")
    comprehensive_analysis_size5()
    
    print("\nNote: Uncomment the comprehensive analysis section to test")
    print("all permutation pairs (warning: this will take significant time!)")


3. COMPREHENSIVE ANALYSIS:
COMPREHENSIVE ANALYSIS FOR SIZE 5 PERMUTATIONS
Total permutations of size 5: 120
Total possible pairs: 14280

Testing all pairs with depth limit 6...
(This may take a while...)
Testing 1000 randomly sampled pairs...
Progress: 0/1000
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 2!
Search completed. Explored 30 nodes.
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 1!
Search completed. Explored 6 nodes.
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 2!
Search completed. Explored 35 nodes.
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 3!
Search completed. Explored 96 nodes.
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 3!
Search completed. Explored 90 nodes.
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with

In [31]:
from itertools import permutations
from collections import deque
import heapq

def generate_single_translocation(perm, start_idx, end_idx, insert_pos):
    """
    Generate a single translocation: move interval [start_idx:end_idx] to insert_pos.
    Returns the resulting permutation.
    """
    # Extract the interval (keeping its internal order)
    interval = perm[start_idx:end_idx]
    # Get the remaining elements
    remaining = perm[:start_idx] + perm[end_idx:]
    # Insert interval at the specified position
    result = remaining[:insert_pos] + interval + remaining[insert_pos:]
    return result

def get_all_possible_translocations(perm):
    """
    Get all possible single translocations from a given permutation.
    Returns list of (result_permutation, move_info) tuples.
    """
    translocations = []
    n = len(perm)
    
    # Try all possible intervals
    for start_idx in range(n):
        for end_idx in range(start_idx + 1, n + 1):
            interval = perm[start_idx:end_idx]
            remaining = perm[:start_idx] + perm[end_idx:]
            
            # Try all possible insertion positions
            for insert_pos in range(len(remaining) + 1):
                result = remaining[:insert_pos] + interval + remaining[insert_pos:]
                
                # Only include if different from original
                if result != perm:
                    move_info = {
                        'interval': interval,
                        'from_pos': (start_idx, end_idx),
                        'to_pos': insert_pos,
                        'description': f"Move '{interval}' from [{start_idx}:{end_idx}] to pos {insert_pos}"
                    }
                    translocations.append((result, move_info))
    
    return translocations

def heuristic_distance(current_perm, target_perm):
    """
    Heuristic function: number of elements not in correct position.
    This is admissible (never overestimates) because each translocation
    can fix at most n-1 positions in the best case.
    """
    return sum(1 for i in range(len(current_perm)) if current_perm[i] != target_perm[i])

def better_heuristic(current_perm, target_perm):
    """
    Better heuristic: minimum number of intervals that need to be moved.
    More informed than simple position counting.
    """
    if current_perm == target_perm:
        return 0
    
    n = len(current_perm)
    # Count longest common subsequences that are already in place
    correctly_placed = 0
    
    # Find the longest subsequence that's already in the right relative positions
    target_positions = {val: i for i, val in enumerate(target_perm)}
    current_positions = {val: i for i, val in enumerate(current_perm)}
    
    # Simple heuristic: count elements that are out of place
    out_of_place = sum(1 for val in current_perm 
                      if current_positions[val] != target_positions[val])
    
    # Each move can potentially fix multiple positions, but conservatively
    # estimate that we need at least ceil(out_of_place / n) moves
    return max(1, out_of_place // n) if out_of_place > 0 else 0

class SearchNode:
    def __init__(self, permutation, path, moves, g_cost, h_cost):
        self.permutation = permutation
        self.path = path  # List of permutations in the path
        self.moves = moves  # List of move descriptions
        self.g_cost = g_cost  # Actual cost (number of moves so far)
        self.h_cost = h_cost  # Heuristic cost (estimated remaining cost)
        self.f_cost = g_cost + h_cost  # Total estimated cost
    
    def __lt__(self, other):
        # For priority queue: prefer lower f_cost, then lower g_cost
        if self.f_cost == other.f_cost:
            return self.g_cost < other.g_cost
        return self.f_cost < other.f_cost

def branch_and_bound_search(start_perm, target_perm, max_depth=10, use_better_heuristic=True):
    """
    Find shortest path using branch and bound with A* strategy.
    """
    if start_perm == target_perm:
        return 0, [start_perm], []
    
    # Choose heuristic function
    heuristic_func = better_heuristic if use_better_heuristic else heuristic_distance
    
    # Priority queue: (f_cost, node_id, node)
    initial_h = heuristic_func(start_perm, target_perm)
    initial_node = SearchNode(start_perm, [start_perm], [], 0, initial_h)
    
    priority_queue = [initial_node]
    heapq.heapify(priority_queue)
    
    visited = {start_perm: 0}  # permutation -> best_g_cost_so_far
    best_solution = None
    nodes_explored = 0
    
    print(f"Starting branch and bound search...")
    print(f"Initial heuristic estimate: {initial_h}")
    
    while priority_queue:
        current_node = heapq.heappop(priority_queue)
        nodes_explored += 1
        
        # Pruning: if we already found a better solution, skip
        if best_solution and current_node.f_cost >= best_solution.g_cost:
            continue
            
        # Depth limiting
        if current_node.g_cost >= max_depth:
            continue
        
        # Progress reporting
        if nodes_explored % 1000 == 0:
            print(f"Explored {nodes_explored} nodes, current best f_cost: {current_node.f_cost}")
        
        # Generate all possible next moves
        possible_moves = get_all_possible_translocations(current_node.permutation)
        
        for next_perm, move_info in possible_moves:
            new_g_cost = current_node.g_cost + 1
            
            # Check if we reached the target
            if next_perm == target_perm:
                solution_node = SearchNode(
                    next_perm,
                    current_node.path + [next_perm],
                    current_node.moves + [move_info['description']],
                    new_g_cost,
                    0
                )
                
                if not best_solution or new_g_cost < best_solution.g_cost:
                    best_solution = solution_node
                    print(f"Found solution with cost {new_g_cost}!")
                continue
            
            # Pruning: skip if we've seen this state with better or equal cost
            if next_perm in visited and visited[next_perm] <= new_g_cost:
                continue
            
            # Pruning: skip if estimated total cost exceeds current best
            new_h_cost = heuristic_func(next_perm, target_perm)
            new_f_cost = new_g_cost + new_h_cost
            
            if best_solution and new_f_cost >= best_solution.g_cost:
                continue
            
            # Add to queue
            visited[next_perm] = new_g_cost
            new_node = SearchNode(next_perm, 
                                current_node.path + [next_perm],
                                current_node.moves + [move_info['description']],
                                new_g_cost, 
                                new_h_cost)
            
            heapq.heappush(priority_queue, new_node)
    
    print(f"Search completed. Explored {nodes_explored} nodes.")
    
    if best_solution:
        return best_solution.g_cost, best_solution.path, best_solution.moves
    else:
        return float('inf'), [], []

def analyze_with_branch_and_bound(start_perm, target_perm, max_depth=8):
    """
    Analyze transformation using branch and bound.
    """
    start_str = ''.join(map(str, start_perm))
    target_str = ''.join(map(str, target_perm))
    
    print(f"BRANCH AND BOUND ANALYSIS: {start_str} → {target_str}")
    print("=" * 60)
    
    # Try with better heuristic first
    print("Using better heuristic...")
    distance, path, moves = branch_and_bound_search(start_perm, target_perm, max_depth, True)
    
    if distance == float('inf'):
        print(f"No solution found within depth limit {max_depth}")
        
        # Try with simple heuristic as backup
        print("\nTrying with simple heuristic...")
        distance, path, moves = branch_and_bound_search(start_perm, target_perm, max_depth, False)
    
    if distance == float('inf'):
        print("No solution found with either heuristic!")
        return
    
    print(f"\nMinimum distance: {distance}")
    print("\nOptimal transformation path:")
    
    for i, perm in enumerate(path):
        perm_str = ''.join(map(str, perm))
        if i == 0:
            print(f"Step {i}: {perm_str} (start)")
        else:
            print(f"Step {i}: {perm_str}")
            print(f"       <- {moves[i-1]}")
    
    return distance, path, moves

def compare_algorithms(start_perm, target_perm):
    """
    Compare BFS vs Branch and Bound
    """
    print("ALGORITHM COMPARISON")
    print("=" * 60)
    
    # BFS (from previous implementation)
    print("1. BFS (Breadth-First Search):")
    try:
        from collections import deque
        # Simple BFS for comparison
        if start_perm == target_perm:
            bfs_distance = 0
        else:
            queue = deque([(start_perm, 0)])
            visited = {start_perm}
            bfs_distance = float('inf')
            
            while queue and bfs_distance == float('inf'):
                current_perm, depth = queue.popleft()
                if depth >= 6:  # Limit for comparison
                    break
                    
                moves = get_all_possible_translocations(current_perm)
                for next_perm, _ in moves:
                    if next_perm == target_perm:
                        bfs_distance = depth + 1
                        break
                    if next_perm not in visited:
                        visited.add(next_perm)
                        queue.append((next_perm, depth + 1))
        
        print(f"   Result: {bfs_distance}")
    except:
        print("   BFS failed")
    
    print("\n2. Branch and Bound with A*:")
    bb_distance, _, _ = branch_and_bound_search(start_perm, target_perm, 6, True)
    print(f"   Result: {bb_distance}")

def generate_random_test_cases(n=5, num_cases=20, seed=42):
    """
    Generate random test cases for permutations of size n.
    """
    import random
    random.seed(seed)
    
    all_perms = list(permutations(range(1, n + 1)))
    test_cases = []
    
    for _ in range(num_cases):
        start = random.choice(all_perms)
        target = random.choice(all_perms)
        # Avoid trivial cases where start == target
        while target == start:
            target = random.choice(all_perms)
        test_cases.append((start, target))
    
    return test_cases

def comprehensive_analysis_size5():
    """
    Comprehensive analysis of all permutation pairs of size 5.
    Warning: This is 5! × 5! = 14,400 pairs, so we'll sample smartly.
    """
    print("COMPREHENSIVE ANALYSIS FOR SIZE 5 PERMUTATIONS")
    print("=" * 60)
    
    all_perms = list(permutations(range(1, 6)))  # All 120 permutations of size 5
    print(f"Total permutations of size 5: {len(all_perms)}")
    print(f"Total possible pairs: {len(all_perms) * (len(all_perms) - 1)}")
    
    # Statistics tracking
    distance_counts = {}
    impossible_count = 0
    total_tested = 0
    max_depth_limit = 6
    
    print(f"\nTesting all pairs with depth limit {max_depth_limit}...")
    print("(This may take a while...)")
    
    # Test all pairs (or sample if too many)
    sample_size = min(1000, len(all_perms) * (len(all_perms) - 1))  # Limit for practicality
    
    import random
    random.seed(42)
    
    # Generate sample pairs
    test_pairs = []
    for start in all_perms[:20]:  # Take first 20 as starting points
        for target in all_perms:
            if start != target:
                test_pairs.append((start, target))
    
    # Shuffle and take sample
    random.shuffle(test_pairs)
    test_pairs = test_pairs[:sample_size]
    
    print(f"Testing {len(test_pairs)} randomly sampled pairs...")
    
    for i, (start, target) in enumerate(test_pairs):
        if i % 100 == 0:
            print(f"Progress: {i}/{len(test_pairs)}")
        
        distance, _, _ = branch_and_bound_search(start, target, max_depth_limit, True)
        total_tested += 1
        
        if distance == float('inf'):
            impossible_count += 1
        else:
            distance_counts[distance] = distance_counts.get(distance, 0) + 1
    
    # Report statistics
    print(f"\nRESULTS FROM {total_tested} TEST CASES:")
    print("-" * 40)
    print(f"Impossible within depth {max_depth_limit}: {impossible_count}")
    
    print("\nDistance distribution:")
    for dist in sorted(distance_counts.keys()):
        count = distance_counts[dist]
        percentage = (count / total_tested) * 100
        print(f"Distance {dist}: {count} cases ({percentage:.1f}%)")
    
    return distance_counts, impossible_count

def analyze_specific_patterns():
    """
    Analyze specific interesting patterns in size 5 permutations.
    """
    print("ANALYZING SPECIFIC PATTERNS FOR SIZE 5")
    print("=" * 60)
    
    patterns = [
        # Identity and simple swaps
        ((1, 2, 3, 4, 5), (1, 2, 3, 4, 5)),  # Identity (should be 0)
        ((1, 2, 3, 4, 5), (2, 1, 3, 4, 5)),  # Adjacent swap
        ((1, 2, 3, 4, 5), (1, 3, 2, 4, 5)),  # Non-adjacent swap
        
        # Rotations
        ((1, 2, 3, 4, 5), (2, 3, 4, 5, 1)),  # Left rotation
        ((1, 2, 3, 4, 5), (5, 1, 2, 3, 4)),  # Right rotation
        
        # Reversals
        ((1, 2, 3, 4, 5), (5, 4, 3, 2, 1)),  # Complete reversal
        ((1, 2, 3, 4, 5), (3, 2, 1, 4, 5)),  # Partial reversal
        
        # Complex patterns
        ((1, 2, 3, 4, 5), (3, 1, 4, 2, 5)),  # Complex rearrangement
        ((1, 2, 3, 4, 5), (4, 5, 1, 2, 3)),  # Block movement
    ]
    
    for i, (start, target) in enumerate(patterns):
        start_str = ''.join(map(str, start))
        target_str = ''.join(map(str, target))
        print(f"\nPattern {i+1}: {start_str} → {target_str}")
        
        if start == target:
            print("Distance: 0 (identity)")
            continue
            
        distance, path, moves = branch_and_bound_search(start, target, 8, True)
        print(f"Distance: {distance}")
        
        if distance != float('inf') and distance <= 4:  # Show path for short distances
            print("Path:")
            for j, perm in enumerate(path):
                perm_str = ''.join(map(str, perm))
                if j == 0:
                    print(f"  Step {j}: {perm_str}")
                else:
                    print(f"  Step {j}: {perm_str} <- {moves[j-1]}")

def find_maximum_distance(n=5, max_search_depth=6, test_all=False):
    """
    Find the maximum translocation distance between any two permutations of size n.
    
    Args:
        n: Size of permutations
        max_search_depth: Maximum depth to search
        test_all: If True, test all pairs (warning: can be very slow!)
    """
    print(f"FINDING MAXIMUM DISTANCE FOR n={n}")
    print("=" * 60)
    
    all_perms = list(permutations(range(1, n + 1)))
    total_perms = len(all_perms)
    total_ordered_pairs = total_perms * (total_perms - 1)
    total_unordered_pairs = total_ordered_pairs // 2
    
    print(f"Total permutations: {total_perms}")
    print(f"Total ordered pairs (A→B ≠ B→A): {total_ordered_pairs:,}")
    print(f"Total unordered pairs: {total_unordered_pairs:,}")
    
    max_distance = 0
    max_distance_pairs = []
    distance_counts = {}
    tested_pairs = 0
    
    if test_all and total_ordered_pairs <= 20000:
        # Test all pairs if requested and manageable
        print(f"\nTesting ALL {total_ordered_pairs:,} ordered pairs...")
        test_pairs = [(start, target) for start in all_perms for target in all_perms if start != target]
        
    elif test_all:
        print(f"\nERROR: {total_ordered_pairs:,} pairs is too many for exhaustive testing!")
        print("Consider using test_all=False for strategic sampling.")
        return None, None, None
        
    else:
        # Strategic sampling
        print(f"\nUsing strategic sampling (not testing all {total_ordered_pairs:,} pairs)...")
        test_pairs = []
        
        # Strategy 1: Include all pairs involving the identity permutation
        identity = tuple(range(1, n + 1))
        for target in all_perms:
            if target != identity:
                test_pairs.append((identity, target))
        
        # Strategy 2: Include all pairs involving the complete reversal
        reversal = tuple(range(n, 0, -1))
        for start in all_perms:
            if start != reversal:
                test_pairs.append((start, reversal))
        
        # Strategy 3: Include pairs between "structured" permutations
        structured_perms = [identity, reversal]
        if n >= 4:
            # Alternating patterns
            alt1 = tuple([i for i in range(1, n + 1, 2)] + [i for i in range(2, n + 1, 2)])
            alt2 = tuple([i for i in range(2, n + 1, 2)] + [i for i in range(1, n + 1, 2)])
            structured_perms.extend([alt1, alt2])
        
        for start in structured_perms:
            for target in all_perms:
                if start != target and (start, target) not in test_pairs:
                    test_pairs.append((start, target))
        
        # Strategy 4: Add random sample of remaining pairs
        import random
        random.seed(42)
        all_remaining = [(start, target) for start in all_perms for target in all_perms 
                        if start != target and (start, target) not in test_pairs]
        
        sample_size = min(2000, len(all_remaining))  # Take up to 2000 additional samples
        test_pairs.extend(random.sample(all_remaining, sample_size))
        
        # Remove duplicates
        test_pairs = list(set(test_pairs))
    
    total_to_test = len(test_pairs)
    coverage_percent = (total_to_test / total_ordered_pairs) * 100
    
    print(f"Testing {total_to_test:,} pairs ({coverage_percent:.1f}% coverage)")
    print(f"Search depth limit: {max_search_depth}")
    print()
    
    # Test each pair
    for i, (start, target) in enumerate(test_pairs):
        if i % 500 == 0:
            print(f"Progress: {i:,}/{total_to_test:,} ({(i/total_to_test)*100:.1f}%), current max: {max_distance}")
        
        distance, _, _ = branch_and_bound_search(start, target, max_search_depth, True)
        tested_pairs += 1
        
        if distance != float('inf'):
            distance_counts[distance] = distance_counts.get(distance, 0) + 1
            
            if distance > max_distance:
                max_distance = distance
                max_distance_pairs = [(start, target)]
                print(f"\n*** NEW MAXIMUM DISTANCE: {max_distance} ***")
                start_str = ''.join(map(str, start))
                target_str = ''.join(map(str, target))
                print(f"    Example: {start_str} → {target_str}")
            elif distance == max_distance:
                max_distance_pairs.append((start, target))
    
    # Report results
    print(f"\n" + "="*60)
    print(f"FINAL RESULTS:")
    print(f"Tested: {tested_pairs:,} pairs ({coverage_percent:.1f}% of all pairs)")
    print(f"Maximum distance found: {max_distance}")
    print(f"Pairs achieving maximum distance: {len(max_distance_pairs)}")
    
    if not test_all:
        print(f"\nNOTE: This is based on {coverage_percent:.1f}% sampling.")
        print(f"For definitive results, set test_all=True")
        print(f"(Warning: will test all {total_ordered_pairs:,} pairs)")
    
    print(f"\nDistance distribution:")
    for dist in sorted(distance_counts.keys()):
        count = distance_counts[dist]
        percentage = (count / tested_pairs) * 100
        print(f"Distance {dist}: {count:,} pairs ({percentage:.1f}%)")
    
    # Show examples of maximum distance pairs
    print(f"\nExamples achieving maximum distance {max_distance}:")
    for i, (start, target) in enumerate(max_distance_pairs[:5]):
        start_str = ''.join(map(str, start))
        target_str = ''.join(map(str, target))
        print(f"{i+1}. {start_str} → {target_str}")
    
    return max_distance, max_distance_pairs, distance_counts

def get_exhaustive_analysis_option():
    """
    Give user choice for exhaustive analysis.
    """
    print("EXHAUSTIVE ANALYSIS OPTIONS FOR n=5:")
    print("=" * 50)
    print("Option 1: Strategic sampling (~3,000 pairs, ~20% coverage)")
    print("Option 2: Exhaustive testing (14,280 pairs, 100% coverage)")
    print("\nNote: Option 2 will take significantly longer but gives definitive results")
    
    return input("Choose option (1 or 2): ").strip()

if __name__ == "__main__":
    print("MAXIMUM DISTANCE ANALYSIS FOR n=5")
    print("=" * 60)
    
    # Show the scale of the problem
    n = 5
    total_perms = 120  # 5!
    total_pairs = total_perms * (total_perms - 1)
    
    print(f"PROBLEM SCALE:")
    print(f"- Permutations of size {n}: {total_perms}")
    print(f"- Total ordered pairs to test: {total_pairs:,}")
    print(f"- Previous run tested: ~1,000 pairs (~7% coverage)")
    
    print("\n" + "="*60)
    
    # Option for user choice (simulated - in practice you'd uncomment one)
    print("ANALYSIS OPTIONS:")
    print("1. Strategic sampling (recommended for initial analysis)")
    print("2. Exhaustive testing (for definitive answer)")
    
    # Default: Strategic sampling
    print("\nRunning strategic sampling analysis...")
    max_dist, max_pairs, dist_counts = find_maximum_distance(n=5, max_search_depth=6, test_all=False)
    
    if max_dist is not None:
        print(f"\nBASED ON STRATEGIC SAMPLING:")
        if max_dist == 3:
            print("The maximum distance appears to be 3")
        elif max_dist == 4:
            print("The maximum distance appears to be 4")
        else:
            print(f"The maximum distance appears to be {max_dist}")
        
        print(f"\nTo get the definitive answer, run with test_all=True")
        print(f"(This will test all {total_pairs:,} pairs)")
        
        # Show detailed example
        if max_pairs:
            print(f"\nDETAILED EXAMPLE OF MAXIMUM DISTANCE CASE:")
            start, target = max_pairs[0]
            analyze_with_branch_and_bound(start, target, 8)

def analyze_hardest_cases(n=5):
    """
    Analyze the cases that are likely to have maximum distance.
    """
    print(f"ANALYZING HARDEST CASES FOR n={n}")
    print("=" * 60)
    
    identity = tuple(range(1, n + 1))
    reversal = tuple(range(n, 0, -1))
    
    candidates = [
        (identity, reversal, "Complete reversal"),
        (reversal, identity, "Reverse to identity"),
    ]
    
    # Add some other potentially hard cases
    if n >= 4:
        # Alternating pattern
        alt1 = tuple([i for i in range(1, n + 1, 2)] + [i for i in range(2, n + 1, 2)])
        alt2 = tuple([i for i in range(2, n + 1, 2)] + [i for i in range(1, n + 1, 2)])
        candidates.extend([
            (identity, alt1, "To alternating pattern"),
            (alt1, identity, "From alternating pattern"),
            (alt1, alt2, "Between alternating patterns"),
        ])
    
    print("Testing candidate hardest cases:")
    
    max_found = 0
    for start, target, description in candidates:
        start_str = ''.join(map(str, start))
        target_str = ''.join(map(str, target))
        
        print(f"\n{description}: {start_str} → {target_str}")
        distance, path, moves = branch_and_bound_search(start, target, 8, True)
        
        if distance != float('inf'):
            print(f"Distance: {distance}")
            max_found = max(max_found, distance)
            
            if distance <= 5:  # Show path for reasonable distances
                print("Optimal path:")
                for i, perm in enumerate(path):
                    perm_str = ''.join(map(str, perm))
                    if i == 0:
                        print(f"  Step {i}: {perm_str}")
                    else:
                        print(f"  Step {i}: {perm_str}")
                        print(f"          <- {moves[i-1]}")
        else:
            print(f"Distance: >8 (search limit exceeded)")
    
    return max_found

if __name__ == "__main__":
    print("MAXIMUM DISTANCE ANALYSIS FOR n=5")
    print("=" * 60)
    
    # First, analyze the hardest known cases
    print("1. ANALYZING KNOWN HARD CASES:")
    hardest_known = analyze_hardest_cases(5)
    
    print("\n" + "=" * 60)
    print("2. SYSTEMATIC SEARCH FOR MAXIMUM:")
    
    # Find the actual maximum distance
    max_dist, max_pairs, dist_counts = find_maximum_distance(5, max_search_depth=6)
    
    print(f"\nCONCLUSION:")
    print(f"Based on analysis, the maximum distance for n=5 appears to be: {max_dist}")
    
    if max_dist == 3:
        print("The maximum distance is 3.")
    elif max_dist == 4:
        print("The maximum distance is 4.")
    else:
        print(f"The maximum distance is {max_dist}.")
    
    # Show one complete example of maximum distance transformation
    if max_pairs:
        start, target = max_pairs[0]
        print(f"\nDETAILED EXAMPLE OF MAXIMUM DISTANCE TRANSFORMATION:")
        analyze_with_branch_and_bound(start, target, 8)

MAXIMUM DISTANCE ANALYSIS FOR n=5
PROBLEM SCALE:
- Permutations of size 5: 120
- Total ordered pairs to test: 14,280
- Previous run tested: ~1,000 pairs (~7% coverage)

ANALYSIS OPTIONS:
1. Strategic sampling (recommended for initial analysis)
2. Exhaustive testing (for definitive answer)

Running strategic sampling analysis...
FINDING MAXIMUM DISTANCE FOR n=5
Total permutations: 120
Total ordered pairs (A→B ≠ B→A): 14,280
Total unordered pairs: 7,140

Using strategic sampling (not testing all 14,280 pairs)...
Testing 2,592 pairs (18.2% coverage)
Search depth limit: 6

Progress: 0/2,592 (0.0%), current max: 0
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 3!
Search completed. Explored 118 nodes.

*** NEW MAXIMUM DISTANCE: 3 ***
    Example: 41253 → 32451
Starting branch and bound search...
Initial heuristic estimate: 1
Found solution with cost 2!
Search completed. Explored 76 nodes.
Starting branch and bound search...
Initial heuristic estima